# Dataset: CIFAR 10

> CIFAR 10


- skip_showdoc: true
- skip_exec: true

In [ ]:
#| default_exp cifar

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
from fastbook import search_images_ddg
from fastdownload import download_url
from fastai.vision.all import *
from nbdevAuto import functions
import os
import shutil